# Red Neuronal Residual (ResNet)

## Autor

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 


## References

1. [Documentación de Keras](https://keras.io/getting-started/sequential-model-guide/)
2. [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/pdf/1502.03167.pdf)
3. [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)
4. [Convolutional Neural Networks at Constrained Time Cost](https://arxiv.org/pdf/1412.1710.pdf)

# Introducción

El problema de las redes profundas (deep neuronal network) es que en la medida que el modelo es mas profundo, el gradiente tiende a desvanecerse (volverse cero).

Para remediar esta situación se han introducido varias soluciones. En esta lección se introduce las redes residuales. Este ha sido un tipo muy exitoso de red profunda.

La idea central es que alguna capas interiores son concetadas con las capas mas interiores

# Normalización por lotes

Con el propósito de acelarar el entrenamiento de las redes y para tratar de evitar el desvanecimiento del gradiente Sergey Ioffe y Sergey Ioffe en la paper  [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/pdf/1502.03167.pdf) introdujeron la técnica de normalizar por lotes las activaciones $x$ de cada capa en una red. 

Previamente la función de activación Relu se volvió popular para prevenir el desvanecimiento de gradiente causado en buena parte por la utilización del sigmoide y la tangente hiperbólica.

De acuerdo con los autores, el hecho que la distibución de las entradas a cada capa de la red cambia en cada paso del algoritmo dificulta y demonra ñla convergencia. La propuesta que ellos muestran que funciona es hacer una normalización de tipo estadístico.

La normalización es hecha por los bloques de entrenamiento que ingresan a cada paso de la actualización del gradiente.


Consideremos un mini lote $\mathcal{B}$ de tamaño $m$. Dado que la normalización se aplica a cada activación de forma independiente, vamos a
centrarsnos en una activación particular $x^{(k)}$. Omitimos $k$ por claridad.


Se tienen $m$ valores de esta activación en el mini lote, $\mathcal{B} = \{x_1,\ldots,m\}$. Denotemos los valores normalizados como  $\hat{x}_1,\ldots,\hat{x}_m$, y sus transformaciones lineales como  $y_1,\ldots,y_m$.

Los autores se refieren a la transformación

$$
BN_{\gamma,\beta}: x_{1,\ldots,m} \to y_{1 \ldots, m},
$$


como la transformación de normalización por lotes. Los valores $\gamma,\beta$ son parámetros que deben ser aprendidos.

La transformación $BN$ es como sigue. En el algortimo, $\epsilon$ es una constante usada para estabilización numérica.

## Algoritmo 1. 

- **Entrada**: Valores de $x$  sobre un mini-lote: $\mathcal{B} = \{x_{1,\ldots,m}\}$. Los parámetros $\gamma,\beta$ deben ser aprendidos.
- **Salida** $y_i = BN_{\gamma,\beta}(x_i)$

$$
\begin{align}
\mu_{\mathcal{B}} &= \frac{1}{m} \sum_{i=1}^{m} x_i\\
\sigma^2_{\mathcal{B}} &= \frac{1}{m} \sum_{i=1}^{m}(x_i- \mu_{\mathcal{B}} )^2\\
\hat{x}_i &= \frac{x_i - \mu_{\mathcal{B}}}{\sqrt{\sigma^2_{\mathcal{B}}+\epsilon}}\\
y_i &= \gamma \hat{x}_i + \beta \equiv BN_{\gamma,\beta}(x_i)
\end{align}
$$



# Aprendizaje residual profundo 

Esta arquitectura de red fue introducida por Kaiming He Xiangyu Zhang Shaoqing Ren Jian Sunen el artículo [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf) para el tratamiento de imágenes. La propuesta consiste en pasar las entradas a una capa de convolución y conbinarlas con la salida de las capas de convolución siguientes(se suman). Para poder hacer la suma es necesario que la salida de la capa tenga la misma dimension de la entrada, por lo que se usa *padding= 'same* en las respectivas capas de convolución.

En caso que la salida de la capa de convolución sea menor, se usa una proyección de $x$ a la  dimension requerida.

La imagen ilustra la diferencia entre la red convolucinada clasica y la de aprendizaje residual.

De acuerdo con lo descrito en la sección anterior, asumimos capas de convolución Conv2D con normalización por mini-lotes (BN) y ativación ReLU. Esto se denota *Conv2D-BN-ReLu*.



<figure>
<center>
<img src="./Imagenes/residual_NN.png" width="800" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Comparación de la red convolucionada clasica y la de aprendizaje residual</p>
</figcaption>
</figure>


Supongamos que $\mathcal{F}(W_l,x)$ es la salida de la última capa de convolución en la imagen, en donde $x$ denota la entrada y $W_l$ el conjunto de pesos de las convoluciones.

En el gráfico la entrada a la primera capa convolucionada se denota $x_{l-2}$  con entrada $x_{l-1}$ a la siguiente capa convolucionada. La salida de la segunda capa *Conv2D-BN-ReLu* se denota $x_{l-1}$.

Para la red residual se tiene que la salida de la segunda capa, antes de la activación es $\mathcal{F}(W_l,x)$ . Esta salida se combina con $x_{l-2}$ así:

$$
y_l = \mathcal{F}(W_l,x) + W_s x_{l-2},
$$

en donde $ W_s$ es la proyección (si se requiere). Si $\mathcal{F}(W_l,x)$ y $x_{l-2}$ tienen la misma dimensión, $W_s$ es la matriz identidad.

Finalmente

$$
x_l = Relu(y_l).
$$

La función  $\mathcal{F}$ se denomina función residual. Es bastante flexible la forma de esta función. En el ejemplo esta conformada por dos capas *Conv2D-BN-ReLu*. Los experimentos muesrean que dos o tres capas funcionan bien. Una sola capa es posible, pero en este caso, se reduce a una simple transformación lineal. 


# Datos  CIFAR10

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

## Carga los datos

In [ ]:
(x_train,y_train), (x_test,y_test) = cifar10.load()

In [ ]:
num_labels = len(unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

image_size = x_train.shape[1]
x_train.reshape([-1,image_size,image_size,1])
x_test.reshape([-1,image_size,image_size,1])

x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

input_shape = (image_size, image_size,1)
batch_size = 12
epochs = 20
kernel_size = 3
filters = 128
dropout = 0.3

inputs = Input(shape=input_shape)
y = Conv2D(filters=filters, kernel_size = kernel_size, 
          activation='relu',)(inputs)
y = MaxPooling(y,2)
y  = Conv2D(filters=filters, kernel_size = kernel_size, 
          activation='relu',)(y)
y = MaxPooling(y,2)
y  = Conv2D(filters=filters, kernel_size = kernel_size, 
          activation='relu',)(y)
y = MaxPooling(y,2)
y = Flatten()(y)
y = Dropout(dropout)(y)
ouputs = Dense(num_labels,activation='softmax')(y)
model = Model(inputs, outputs)

model.compile(loss='categorical_crossentropy',
             optimize ='adam',
             metrics=[accuracy])

history=model.fit(x_train, y_train,
         validation_data= (x_test,y_test)
         epochs= epochs, batch_size=batch_size)
score = model.evaluate(x_test,y_test, 
                       batch_size=batch_size,
                       verbose =0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# numpy package
import numpy as np
import math

# keras mnist module
from keras.datasets import cifar10

# for plotting
import matplotlib.pyplot as plt


# load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

class_id = 0
class_count = 0
images = None
for i in range(100):
    while True:
        index = np.random.randint(0, x_train.shape[0], size=1)
        image = x_train[index]
        if y_train[index] == class_id:
            break

    if images is None:
        images = image
    else:
        images = np.concatenate([images, image], axis=0)
    class_count += 1
    if class_count == 10:
        class_id += 1
        class_count = 0
      
print(images.shape)

plt.figure(figsize=(10, 10))
num_images = images.shape[0]
image_size = images.shape[1]
rows = int(math.sqrt(num_images))
row_names = ['{}'.format(row) for row in ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']]
index = 0
for i in range(num_images):
    ax = plt.subplot(rows, rows, i + 1)
    image = images[i, :, :, :]
    image = np.reshape(image, [image_size, image_size, 3])
    plt.imshow(image)
    # plt.axis('off')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(False)
    ax.xaxis.set_ticks_position('none') 
    ax.yaxis.set_ticks_position('none') 
    if (i % rows) == 0:
        ax.set_ylabel(row_names[index], rotation=45, size='large')
        ax.yaxis.labelpad = 20
        print(row_names[index])
        index += 1

# plt.tight_layout()
plt.savefig("cifar10-samples.png")
plt.show()

# Dense net

In [ ]:
"""Trains a 100-Layer DenseNet on the CIFAR10 dataset.

With data augmentation:
Greater than 93.55% test accuracy in 200 epochs
225sec per epoch on GTX 1080Ti

Densely Connected Convolutional Networks
https://arxiv.org/pdf/1608.06993.pdf
http://openaccess.thecvf.com/content_cvpr_2017/papers/
    Huang_Densely_Connected_Convolutional_CVPR_2017_paper.pdf
Network below is similar to 100-Layer DenseNet-BC (k=12)
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dropout
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import os
import numpy as np

# training parameters
batch_size = 32
epochs = 200
data_augmentation = True

# network parameters
num_classes = 10
num_dense_blocks = 3
use_max_pool = False

# DenseNet-BC with dataset augmentation
# Growth rate   | Depth |  Accuracy (paper)| Accuracy (this)      |
# 12            | 100   |  95.49%          | 93.74%               |
# 24            | 250   |  96.38%          | requires big mem GPU |
# 40            | 190   |  96.54%          | requires big mem GPU |
growth_rate = 12
depth = 100
num_bottleneck_layers = (depth - 4) // (2 * num_dense_blocks)

num_filters_bef_dense_block = 2 * growth_rate
compression_factor = 0.5

# load the CIFAR10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# input image dimensions
input_shape = x_train.shape[1:]

# mormalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


# start model definition
# densenet CNNs (composite function) are made of BN-ReLU-Conv2D
inputs = Input(shape=input_shape)
x = BatchNormalization()(inputs)
x = Activation('relu')(x)
x = Conv2D(num_filters_bef_dense_block,
           kernel_size=3,
           padding='same',
           kernel_initializer='he_normal')(x)
x = concatenate([inputs, x])

# stack of dense blocks bridged by transition layers
for i in range(num_dense_blocks):
    # a dense block is a stack of bottleneck layers
    for j in range(num_bottleneck_layers):
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(4 * growth_rate,
                   kernel_size=1,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(growth_rate,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        x = concatenate([x, y])

    # no transition layer after the last dense block
    if i == num_dense_blocks - 1:
        continue

    # transition layer compresses num of feature maps and reduces the size by 2
    num_filters_bef_dense_block += num_bottleneck_layers * growth_rate
    num_filters_bef_dense_block = int(num_filters_bef_dense_block * compression_factor)
    y = BatchNormalization()(x)
    y = Conv2D(num_filters_bef_dense_block,
               kernel_size=1,
               padding='same',
               kernel_initializer='he_normal')(y)
    if not data_augmentation:
        y = Dropout(0.2)(y)
    x = AveragePooling2D()(y)


# add classifier on top
# after average pooling, size of feature map is 1 x 1
x = AveragePooling2D(pool_size=8)(x)
y = Flatten()(x)
outputs = Dense(num_classes,
                kernel_initializer='he_normal',
                activation='softmax')(y)

# instantiate and compile model
# orig paper uses SGD but RMSprop works better for DenseNet
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(1e-3),
              metrics=['accuracy'])
model.summary()
plot_model(model, to_file="cifar10-densenet.png", show_shapes=True)

# prepare model model saving directory
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_densenet_model.{epoch:02d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# prepare callbacks for model saving and for learning rate reducer
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# run training, with or without data augmentation
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # preprocessing  and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(x_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1,
                        callbacks=callbacks)

# score trained model
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])